<a href="https://colab.research.google.com/github/Zumo09/Feedback-Prize/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the entire repo.
!git clone -l -s git://github.com/Zumo09/Feedback-Prize.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 16216, done.
remote: Counting objects: 100% (16216/16216), done.
remote: Compressing objects: 100% (16060/16060), done.
remote: Total 16216 (delta 340), reused 15926 (delta 132), pack-reused 0
Receiving objects: 100% (16216/16216), 143.50 MiB | 25.36 MiB/s, done.
Resolving deltas: 100% (340/340), done.
Checking out files: 100% (15640/15640), done.
/content/cloned-repo
class_correlation.ipynb  engine.py  main.py    prova_main.ipynb  util
csv			 input	    models     README.md
datasets		 LICENSE    old_utils  training.ipynb


In [2]:
!pip install transformers
from transformers import LEDTokenizerFast

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

     |████████████████████████████████| 3.5 MB 8.7 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
     |████████████████████████████████| 6.8 MB 38.5 MB/s 
     |████████████████████████████████| 895 kB 57.1 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
tokenizer = LEDTokenizerFast.from_pretrained("allenai/led-base-16384")
df = pd.read_csv('./input/feedback-prize-2021/train.csv')
df["token_prediction"] = None

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

In [4]:
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,token_prediction
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,None
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,None
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,None
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,None
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,None
...,...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...,None
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...,None
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838,None
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...,None


In [5]:
for i in tqdm(range(len(df))):
  id = df.iloc[i]["id"]
  discourse_start = int(df.iloc[0]["discourse_start"])
  discourse_end = int(df.iloc[0]["discourse_end"])

  with open(f'./input/feedback-prize-2021/train/{id}.txt', 'r') as f:
    text = f.read()

  char_array = np.zeros( (len(text)) )
  char_array[discourse_start : discourse_end] = 1

  encoding = tokenizer(text, padding='max_length', 
                      truncation=True, return_offsets_mapping=True)
  map_token_to_char = encoding['offset_mapping']
  token_array = np.zeros(len(map_token_to_char))
  for k in range(len(token_array)):
    token_array[k] = char_array[ map_token_to_char[k][0] - 1]


  df.at[i, "token_prediction"] = " ".join(token_array.nonzero()[0].astype(str)).strip()

  0%|          | 0/144293 [00:00<?, ?it/s]

In [6]:
df

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring,token_prediction
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 ...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59,5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 ...
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75,5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 ...
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...,5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 ...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...,5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 ...
...,...,...,...,...,...,...,...,...,...
144288,4C471936CD75,1.618153e+12,2234.0,3203.0,if I'm not sure what college I want to attend...,Evidence,Evidence 2,386 387 388 389 390 391 392 393 394 395 396 39...,3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20...
144289,4C471936CD75,1.618153e+12,3221.0,4509.0,seeking multiple opinions before making a har...,Evidence,Evidence 3,576 577 578 579 580 581 582 583 584 585 586 58...,3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20...
144290,4C471936CD75,1.618025e+12,4510.0,4570.0,it is better to seek multiple opinions instead...,Position,Position 1,828 829 830 831 832 833 834 835 836 837 838,3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20...
144291,4C471936CD75,1.618025e+12,4570.0,4922.0,The impact of asking people to help you make a...,Evidence,Evidence 4,839 840 841 842 843 844 845 846 847 848 849 85...,3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20...


In [8]:
np.any(df["token_prediction"] == None)

False

In [9]:
df.to_csv("train.csv")